In [8]:
# all imported libraries
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'
import numpy
import pandas as pd
import matplotlib
import csv
import os
import torch
import torchvision
from torchvision import transforms
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, pipeline, ResNetForImageClassification, AutoImageProcessor, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict, load_metric, Image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input, concatenate, Embedding, Conv1D, GlobalMaxPooling1D, TextVectorization
from tensorflow.keras.optimizers import Adam
import PIL
from PIL import Image as im
from PIL import ImageOps as imops
import shutil

In [ ]:
!grep Instagram_Username_Dataset_Updated.csv

^C


####Data to Array

In [2]:
usernames = []
followers = []
following = []
biography = []
profile_picture = []
post_image = []
post_caption = []
labels = []
count = 0

__file__ = "ECS271.ipynb"
script_dir = os.path.dirname(__file__)
script_dir = os.path.join(script_dir, "data")

print("Filling in Usernames")
# fill in all usernames and their label
with open('./Instagram_Username_Dataset_Updated.csv', encoding='utf-8') as fp:
    csvreader = csv.reader(fp)
    for name in csvreader:
        file_path = os.path.join(script_dir, name[0])
        follow_path = os.path.join(file_path, name[0] + "_follower_info.txt")
        if (os.path.isfile(follow_path) == False):
            continue
        usernames.append(name[0])
        labels.append(int(name[1]))

# loop through every user data file=
for name in usernames:
    file_path = os.path.join(script_dir, name)
    follow_path = os.path.join(file_path, name + "_follower_info.txt")
    # add numerical data
    try:
        infile = open(follow_path, "r")
        temp = []
        for line in infile:
            line=line.strip()
            line = int(line)
            temp.append(line)
        infile.close()
    except:
        temp = [0, 0]
    followers.append(int(temp[0]))
    following.append(int(temp[1]))

    # add biography data
    bio_path = os.path.join(file_path, name + "_biography.txt")
    temp = ''
    try:
        infile = open(bio_path, "r")
        for line in infile:
            line = line.strip()
            temp = temp + " " + line
    except:
        temp = ''
    biography.append(temp)
    infile.close()

    # add profile image data
    contain_profile = False
    try:
        for images in os.listdir(file_path):
            if (images.endswith("_profile_pic.jpg")):
                profile_picture.append("./data/" + name + "/" + images)
                contain_profile = True
    except:
        contain_profile = False
    if contain_profile == False:
        profile_picture.append('./data/2018-11-21_19-35-46_UTC_profile_pic.jpg') # load in default pfp if missing

    # add post image data
    post_image_path = os.path.join(file_path, name + "_images")
    # add post image data
    temp = []
    contain_posts = False
    has_four = False
    count = 0
    try:
        for images in os.listdir(post_image_path):
            if (images.endswith(".jpg")):
                temp.append(post_image_path + "/" + images)
                contain_posts = True
                if count == 4:
                    has_four = True
                count += 1
    except:
        contain_posts = False
        temp = []
    if contain_posts == False:
        dst = './data/2018-11-21_19-35-46_UTC_profile_pic.jpg'
        temp.append(dst)
        count += 1
    if has_four == False:
        while count < 4:
            dst = './data/2018-11-21_19-35-46_UTC_profile_pic.jpg'
            temp.append(dst)
            count += 1

    post_image.append(temp)

    # add caption image data
    caption_path = os.path.join(file_path, name + "_captions.txt")
    temp = []
    try:
        infile = open(caption_path, "r")
        for line in infile:
            line = line.strip()
            temp.append(line)
    except:
        temp = []
    post_caption.append(temp)
    infile.close()

Filling in Usernames


In [3]:
# append all to train data
train_data = []
pfp_train_init = []
for x in range (0,len(usernames)):
  temp = [usernames[x], followers[x], following[x], biography[x], profile_picture[x], post_image[x], post_caption[x], labels[x]]
  train_data.append(temp)
  #temp = [profile_picture[x], labels[x]]
  #pfp_train_init.append(temp)

# train_data[x][0] = username
# train_data[x][1] = number of followers
# train_data[x][2] = number of users following
# train_data[x][3] = biography
# train_data[x][4] = string of profile picture
# train_data[x][5] = array of strings of post images
# train_data[x][6] = array of post captions
# train_data[x][7] = binary classification

# convert to df
df = pd.DataFrame(train_data)
#df_pfp = pd.DataFrame(pfp_train_init.append)
df.columns = ["username", "followers", "following", "biography", "profile_picture", "post_images", "post_captions", "label"]
#df_pfp.columns = ["profile_picture","label"]

print(df)

# split training and testing data
x = df.drop("label", axis=1)
y = df["label"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

           username  followers  following  \
0         r.zsyifaa        510        372   
1    shabitanurch31        232        242   
2         tatiazulp         25         31   
3             00td4        261        316   
4         lariybaby        613        585   
..              ...        ...        ...   
162        cppijazc          9          8   
163       kcqaw2024         16          8   
164       payxc2024         17          8   
165         ewsfkrd          9          8   
166         phoovow         11          8   

                                             biography  \
0                                            Bismillah   
1                                                        
2     🎀 | romanticizing life ꒰⁠⑅⁠ᵕ⁠༚⁠ᵕ⁠꒱⁠˖⁠♡ @xsven...   
3                                                        
4                       Asuna🐾 • 𝑒𝓈𝒸𝒶𝓅𝑒 𝓉𝒽𝑒 𝑜𝓇𝒹𝒾𝓃𝒶𝓇𝓎 •   
..                                                 ...   
162                                                  

####Numerical Classification

*   Number of Followers
*   Number of Users Following





In [4]:
# function to process numerical attributes
def process_num_attributes(train, test):
	continuous = ["followers", "following"]
	cs = MinMaxScaler()
	train_cont = cs.fit_transform(train[continuous])
	test_cont = cs.transform(test[continuous])
	return train_cont, test_cont

# apply function on data
x_num_train, x_num_test = process_num_attributes(x_train, x_test)

In [5]:
# mlp created with keras
def create_mlp(dim, regress=False):
    input_layer = Input(shape=(dim,))
    # add dense layers
    layer = Dense(8, activation="relu")(input_layer)
    layer = Dense(4, activation="relu")(layer)
    if regress:
        layer = Dense(1, activation="linear")(layer)
    model = Model(inputs=input_layer, outputs=layer)
    return model

# create/compile model
num_input = Input(shape=(x_num_train.shape[1],))
num_test = Input(shape=(x_num_test.shape[1],))
num_model = create_mlp(2, regress=True)

####Image Classification:

*   Profile Picture
*   Post Images



In [6]:
#tensorflow pfp_classifier set-up

pfps = []
post_pics = []
lbls = []
for x in range (0,len(usernames)):
    if profile_picture[x] != '':
        pfps.append(profile_picture[x])
        lbls.append(labels[x])
    else:
        print(usernames[x])

    im_temp = []
    for i in range (4):
        if(len(post_image[x]) > i):
            img = im.open(post_image[x][i])
            img = img.resize((512,512), im.LANCZOS)
            im_temp.append(img)

    im12 = im.new('RGB', (im_temp[0].width + im_temp[1].width, im_temp[0].height))
    im34 = im.new('RGB', (im_temp[2].width + im_temp[3].width, im_temp[2].height))
    im12.paste(im_temp[0], (0, 0))
    im12.paste(im_temp[1], (im_temp[0].width, 0))
    im34.paste(im_temp[2], (0, 0))
    im34.paste(im_temp[3], (im_temp[2].width, 0))

    new_image = im.new('RGB', (im12.width, im12.height + im34.height))
    new_image.paste(im12, (0, 0))
    new_image.paste(im34, (0, im12.height))

    new_image = new_image.resize((224,224), im.LANCZOS)
    img_path = script_dir+"/" + usernames[x] + "/joined_img.jpg"
    new_image.save(img_path)
    post_pics.append(img_path)


In [9]:
#tensorflow pfp_classifier set-up p2
import tensorflow as tf

# Define a function to load and preprocess images
def load_and_preprocess_image(path):
    # Read the image file
    print(path["path"])
    image = tf.io.read_file(path["path"])
    # Decode the image
    image = tf.image.decode_jpeg(image, channels=3)
    # Resize the image to a fixed size (e.g., 224x224)
    image = tf.image.resize(image, [224, 224])
    # Normalize the image to [0, 1] range
    image = image / 255.0
    return dict(image = image)

# Apply the function to the dataset
dataset = dict(path = pfps, label = lbls)
post_dataset = dict(path = post_pics, label = lbls)

dataset = Dataset.from_dict(dataset)
post_dataset = Dataset.from_dict(post_dataset)

dataset = dataset.map(load_and_preprocess_image)
post_dataset = post_dataset.map(load_and_preprocess_image)

#dataset_size = len(pfps)
#train_size = int(0.8 * dataset_size)  # 80% for training
#val_size = dataset_size - train_size  # Remaining 20% for validation




image_train, image_test, image_label_train, image_label_test = train_test_split(dataset["image"], dataset["label"], test_size=0.2, random_state=42)
post_train, post_test, post_label_train, post_label_test = train_test_split(post_dataset["image"], post_dataset["label"], test_size=0.2, random_state=42)
# Split the dataset
#train_dataset = dataset.take(train_size)
#val_dataset = dataset.skip(train_size)

# Shuffle, batch, and prefetch the datasets
#train_dataset = train_dataset.shuffle(buffer_size=1000).batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
#val_dataset = val_dataset.batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

Map:   0%|          | 0/167 [00:00<?, ? examples/s]

./data/r.zsyifaa/2024-04-14_15-08-16_UTC_profile_pic.jpg
./data/shabitanurch31/2024-05-04_13-12-01_UTC_profile_pic.jpg
./data/tatiazulp/2024-02-05_03-39-44_UTC_profile_pic.jpg
./data/00td4/2023-07-30_22-25-56_UTC_profile_pic.jpg
./data/lariybaby/2024-04-22_01-51-40_UTC_profile_pic.jpg
./data/neseekilic/2024-04-25_20-42-22_UTC_profile_pic.jpg
./data/gacha_editsfork/2024-05-01_15-48-31_UTC_profile_pic.jpg
./data/_rawann10_/2024-04-19_11-07-20_UTC_profile_pic.jpg
./data/honeysoymilktea/2024-05-06_22-52-37_UTC_profile_pic.jpg
./data/rarelykind/2024-02-28_21-46-22_UTC_profile_pic.jpg
./data/nsashleyx/2024-05-11_23-44-28_UTC_profile_pic.jpg
./data/ms_grayned/2022-05-29_20-30-31_UTC_profile_pic.jpg
./data/ilovefashion719/2024-05-01_18-27-54_UTC_profile_pic.jpg
./data/5star.iha/2018-11-21_19-35-46_UTC_profile_pic.jpg
./data/ranniiyyaa/2024-03-29_14-32-57_UTC_profile_pic.jpg
./data/kevon.365/2023-06-25_03-56-54_UTC_profile_pic.jpg
./data/hennbro1/2014-11-26_02-59-16_UTC_profile_pic.jpg
./data/i

Map:   0%|          | 0/167 [00:00<?, ? examples/s]

data/r.zsyifaa/joined_img.jpg
data/shabitanurch31/joined_img.jpg
data/tatiazulp/joined_img.jpg
data/00td4/joined_img.jpg
data/lariybaby/joined_img.jpg
data/neseekilic/joined_img.jpg
data/gacha_editsfork/joined_img.jpg
data/_rawann10_/joined_img.jpg
data/honeysoymilktea/joined_img.jpg
data/rarelykind/joined_img.jpg
data/nsashleyx/joined_img.jpg
data/ms_grayned/joined_img.jpg
data/ilovefashion719/joined_img.jpg
data/5star.iha/joined_img.jpg
data/ranniiyyaa/joined_img.jpg
data/kevon.365/joined_img.jpg
data/hennbro1/joined_img.jpg
data/inoka_ath/joined_img.jpg
data/verra_uliani/joined_img.jpg
data/rebeccasugar/joined_img.jpg
data/danaterrace/joined_img.jpg
data/baleen.sound/joined_img.jpg
data/xoxomobxoxo/joined_img.jpg
data/sthamtp/joined_img.jpg
data/gwagwa._/joined_img.jpg
data/hayeyashi/joined_img.jpg
data/alayna.smith120/joined_img.jpg
data/fakedgold/joined_img.jpg
data/aw_yes/joined_img.jpg
data/cat.finay/joined_img.jpg
data/theicon5/joined_img.jpg
data/elijah__self/joined_img.jpg
da

In [60]:
print(post_train[0])

In [11]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam


#RESNET-50 model pre-trained and fine-tunable
image_inputs = Input(shape=(224,224,3))
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

for layer in base_model.layers[-10:]:  # Adjust the number of layers to unfreeze as needed
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Example of a fully connected layer
predictions = Dense(1, activation='softmax')(x)  # Change `num_classes` to your number of classes

pfp_model = Model(inputs=base_model.input, outputs=predictions)
pfp_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
pfp_model.fit(image_train, image_label_train, epochs=10, verbose=0)


post_model = Model(inputs=base_model.input, outputs=predictions)
post_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
post_model.fit(post_train, post_label_train, epochs=10, verbose=0)


Yippeee!


####Text Classification:

*   Usernames
*   Biography
*   Post Captions



In [12]:
#ADD FOLLOWING:
#Edit tokenization
#Create train, and dev set
def list_to_dict(data, labeled):
    sentences1 = []
    sentences2 = []
    sentences3 = []
    labels = []
    for item in data:
        sentences1.append(item[0])
        sentences2.append(item[3])
        sentences3.append(item[6])
        if labeled:
            labels.append(item[7])
    if labeled:
        convert_dict = dict(sentence1 = sentences1, sentence2 = sentences2, sentence3 = sentences3, label = labels)
    else:
        convert_dict = dict(sentence1 = sentences1, sentence2 = sentences2, sentences3 = sentences3)
    return convert_dict


#Tokenizes the dataset, by creating the input_id from tokenizing the two sentences
def tokenize_function(dataset):
    captionlist = []
    for captions in range(0,len(dataset["sentence1"])):
        captionlist.append(dataset["sentence1"][captions] + "[SEP]")
    for captions in range(0,len(dataset["sentence2"])):
        captionlist[captions] += dataset["sentence2"][captions] + "[SEP]"
    for captions in range(0,len(dataset["sentence3"])):
        tempcaptions = ""
        for caption in dataset["sentence3"][captions]:
            tempcaptions += caption
        captionlist[captions] += tempcaptions
    #Change to usernames [0], biography [3], post captions [6]
    return captionlist
    return dict(tokenizer(dataset["sentence1"], dataset["sentence2"], captionlist, truncation=True, padding=True, return_tensors="np"))


total_set = Dataset.from_dict(list_to_dict(train_data, True))
labels = total_set["label"]

max_features = 200000
embedding_dim = 128
sequence_length = 500

vectorize_layer = TextVectorization(
    max_tokens = max_features,
    output_mode = "int",
    output_sequence_length = sequence_length
)


text_train, text_val, label_train, label_val = train_test_split(total_set, labels, test_size=0.2, random_state=42)


#text_train = text_train.map(vectorize_text)

text_inputs = Input(shape=(1,), dtype="string")
x = vectorize_layer(text_inputs)
x = Embedding(max_features, embedding_dim)(x)
x = Dropout(0.5)(x)
x = Conv1D(128, 7, padding = "valid", activation = "relu", strides = 3) (x)
x = Conv1D(128, 7, padding = "valid", activation = "relu", strides = 3) (x)
x = GlobalMaxPooling1D() (x)
x = Dense(128, activation = "relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation = "sigmoid", name = "outputs")(x)
text_model = Model(text_inputs, outputs)
#Finetune RoBERTa large with the training data
#text_model = TFAutoModelForSequenceClassification.from_pretrained("FacebookAI/roberta-large", num_labels=2)
#tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")

#Turn training, dev, and test set into datasets
#Tokenize the datasets for processing
tokenized_train_dataset = pd.DataFrame(data = tokenize_function(text_train))
tokenized_train_dataset.col = ["text"]
tokenized_val_dataset = pd.DataFrame(data = tokenize_function(text_val))
tokenized_val_dataset.col = ["text"]
vectorize_layer.adapt(tokenized_train_dataset)
#tokenized_dev_dataset = dev_dataset.map(tokenize_function, batched=True)

text_model.compile(optimizer="adam", metrics=["accuracy"], loss = "binary_crossentropy")  # No loss argument!

text_model.fit([tokenized_train_dataset], y_train, epochs=10)

C:\Users\Aaron Nguyen\AppData\Local\Temp\ipykernel_21440\2925406822.py:76: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  tokenized_train_dataset.col = ["text"]
C:\Users\Aaron Nguyen\AppData\Local\Temp\ipykernel_21440\2925406822.py:78: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  tokenized_val_dataset.col = ["text"]



Epoch 1/10
5/5 [==============================] - 3s 194ms/step - loss: 0.6625 - accuracy: 0.7068
Epoch 2/10
5/5 [==============================] - 1s 193ms/step - loss: 0.5513 - accuracy: 0.8872
Epoch 3/10
5/5 [==============================] - 1s 194ms/step - loss: 0.4172 - accuracy: 0.9098
Epoch 4/10
5/5 [==============================] - 1s 196ms/step - loss: 0.2875 - accuracy: 0.9098
Epoch 5/10
5/5 [==============================] - 1s 190ms/step - loss: 0.2445 - accuracy: 0.9173
Epoch 6/10
5/5 [==============================] - 1s 189ms/step - loss: 0.2420 - accuracy: 0.9398
Epoch 7/10
5/5 [==============================] - 1s 191ms/step - loss: 0.2016 - accuracy: 0.9398
Epoch 8/10
5/5 [==============================] - 1s 189ms/step - loss: 0.2317 - accuracy: 0.9398
Epoch 9/10
5/5 [==============================] - 1s 188ms/step - loss: 0.1845 - accuracy: 0.9398
Epoch 10/10
5/5 [==============================] - 1s 192ms/step - loss: 0.2018 - accuracy: 0.9474


####Multimodal Neural Network Model

Combine all features and feed through single model



In [22]:
post_inputs = Input(shape=(224,224,3))
# concatenate all separate inputs and process
combined = concatenate([num_model(num_input), text_model(text_inputs), pfp_model(image_inputs), post_model(post_inputs)])
z = Dense(10, activation="relu")(combined)
z = Dense(1, activation="sigmoid")(z)

# final model
model = Model(inputs=[num_input, text_inputs, image_inputs, post_inputs], outputs=z)

image_train = numpy.asarray(image_train)
image_test = numpy.asarray(image_test)
post_train = numpy.asarray(post_train)
post_test = numpy.asarray(post_test)

# training
model.compile(loss="binary_crossentropy", metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model.fit([x_num_train, tokenized_train_dataset, image_train, post_train], y_train, epochs=20, batch_size=8, verbose=2)

# evaluate based on loss and accuracy
loss, accuracy, precision, recall = model.evaluate([x_num_test, tokenized_val_dataset, image_test, post_test], y_test, verbose=2)
f1 = 2* precision*recall/(precision + recall)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")
print(f"f1-loss: {f1}")

Epoch 1/20
17/17 - 33s - loss: 0.6521 - accuracy: 0.5263 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - 33s/epoch - 2s/step
Epoch 2/20
17/17 - 10s - loss: 0.6320 - accuracy: 0.5263 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - 10s/epoch - 587ms/step
Epoch 3/20
17/17 - 10s - loss: 0.6200 - accuracy: 0.5263 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - 10s/epoch - 590ms/step
Epoch 4/20
17/17 - 11s - loss: 0.6133 - accuracy: 0.6316 - precision_5: 1.0000 - recall_5: 0.2222 - 11s/epoch - 622ms/step
Epoch 5/20
17/17 - 10s - loss: 0.6069 - accuracy: 0.9549 - precision_5: 0.9130 - recall_5: 1.0000 - 10s/epoch - 614ms/step
Epoch 6/20
17/17 - 11s - loss: 0.6010 - accuracy: 0.9549 - precision_5: 0.9130 - recall_5: 1.0000 - 11s/epoch - 638ms/step
Epoch 7/20
17/17 - 11s - loss: 0.5944 - accuracy: 0.9549 - precision_5: 0.9130 - recall_5: 1.0000 - 11s/epoch - 633ms/step
Epoch 8/20
17/17 - 11s - loss: 0.5881 - accuracy: 0.9549 - precision_5: 0.9130 - recall_5: 1.0000 - 11s/epoch - 633ms/

In [ ]:
# create prediction from model
y_pred = model.predict([x_num_test, x_text_test])
y_pred_classes = (y_pred > 0.5).astype("int32")  # Convert probabilities to binary class predictions

# print report
print(classification_report(y_test, y_pred_classes))